In [1]:
import os
import numpy as np
import pandas as pd
import pymc3 as pm
from scipy import stats

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from dataloader import *

sns.set(style='ticks', context='talk')
plt.style.use("paper.mplstyle")

np.random.seed(1032020)

In [2]:
size = 100
true_intercept = 1
true_slope = 2

# x1 = stats.norm(1, 1).rvs(1000)
# x2 = stats.norm(2, 1).rvs(1000)
# x3 = stats.norm(3, 1).rvs(1000)

x_true = np.linspace(0, 1, size) 
y_true = true_intercept + x_true * true_slope
x1 = np.random.normal(loc=x_true, scale=0.2, size=(500, len(y_true)))
x2 = np.random.normal(loc=x_true+1, scale=0.2, size=(500, len(y_true)))
x = np.stack((x1, x2)).reshape(len(y_true), len(x1)+len(x2))


sigma_y = np.abs(np.random.normal(loc=0.5, scale=0.1, size=len(y_true)))
y_err = np.random.normal(scale=sigma_y, size=len(y_true))

y = y_true + y_err

display(pd.DataFrame({"y":y}).head())
# plt.errorbar(x, y, yerr=sigma_y, fmt='ko', lw=0.5, ms=5)
# plt.plot(x, y_true)

,y
0,1.340948
1,0.542309
2,2.210151
3,0.924678
4,1.056327


In [ ]:
with pm.Model() as model:
    intercept = pm.Uniform('intercept', 0, 5)
    slope = pm.Uniform('slope', 0, 5)
#     _sigma_x = pm.Uniform('sigma_x', 0, 1, shape=y.shape[0])
    _sigma_y = pm.Uniform('sigma_y', 0, 1, shape=sigma_y.shape[0], observed=sigma_y)
    scatter = pm.Uniform('scatter', 0, 5)
    total_sigma = pm.Deterministic('sigma', pm.math.sqrt(_sigma_y**2 + scatter))
    
#     _x_true = pm.Uniform('x_true', 0, 1)
    weight = pm.Dirichlet('w', a=np.array([1, 1]))
    g1 = pm.Normal.dist(mu=x.mean(axis=1)-0.5, sigma=0.2)
    g2 = pm.Normal.dist(mu=x.mean(axis=1)+0.5, sigma=0.2)

    likelihood_x = pm.Mixture('x', w=weight, comp_dists=[g1, g2], observed=x.T, shape=y.shape[0])
#     likelihood_x = pm.Normal('x', mu=x.mean(axis=1), sigma=x.std(axis=1), shape=y.shape[0])
    
    _y_true = pm.Deterministic('y_true', slope * likelihood_x + intercept)
    likelihood_y = pm.Normal('y', mu=_y_true, sigma=total_sigma, observed=y)
    
    trace = pm.sample(1000, tune=2000, chains=2)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [w, scatter, slope, intercept]


In [ ]:
from corner import corner

labels = ["intercept", "slope"]
fig = plt.figure(figsize=(10, 10))
corner(np.vstack([trace[k] for k in labels]).T, labels=labels, show_titles=True, truths=[true_intercept, true_slope], quantiles=stats.norm.cdf([-1, 0, 1]), fig=fig);

# fig = plt.figure()
# pm.traceplot(trace)

In [ ]:
# plt.errorbar(x, y, yerr=sigma_y, fmt='ko', lw=0.5, ms=5)

xrange = np.linspace(0, 1, 1000)
idx = np.random.randint(0, len(trace), 500)

plt.plot(xrange, true_intercept + true_slope * xrange, label="True", zorder=100)
for i in idx:
    slope_est = trace.get_values('slope')[i]
    intercept_est = trace.get_values('intercept')[i]
    plt.plot(xrange, xrange*slope_est + intercept_est, color='k', lw=0.5)


plt.legend();